In [19]:
import numpy as np
m=9
n=9
tab=np.zeros([m,n],dtype=int)
obs=np.zeros([m,n],dtype=int)
obs[:,0]=1
obs[0,3:n-2]=1
obs[5:7,3:5]=1
obs[1:3,6]=1
tab[obs==1]=-1

In [20]:
tab

array([[-1,  0,  0, -1, -1, -1, -1,  0,  0],
       [-1,  0,  0,  0,  0,  0, -1,  0,  0],
       [-1,  0,  0,  0,  0,  0, -1,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0, -1, -1,  0,  0,  0,  0],
       [-1,  0,  0, -1, -1,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0]])